# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [65]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [66]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [67]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    word_counts = Counter(text)
    #print(word_counts)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    #print("sorted_vocab")
    #print(sorted_vocab)
    int_to_vocab = { ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    #print("test")
    #print(list(int_to_vocab.items())[:5])
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [68]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_dict = {'.': '||PERIOD||',
              ',': '||COMMA||',
              '"': '||QUOTATION_MARK||',
              ';': '||SEMICOLON||',
              '!': '||EXCLAMATION_MARK||',
              '?': '||QUESTION_MARK||',
              '(': '||LEFT_PAREN||',
              ')': '||RIGHT_PAREN||',
              '--': '||DASH||',
              '\n': '||RETURN||'
              }
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [69]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [70]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [71]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


C:\Users\melin\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [72]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    return input, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [73]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    num_lstm_layers = 1
    keep_prob = 0.8
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    dropout = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([dropout]* num_lstm_layers)
    init_state = tf.identity(cell.zero_state(batch_size, tf.float32), name='initial_state')
    
    return (cell, init_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [74]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed
    """
    
    return tf.contrib.layers.embed_sequence(input_data, vocab_size, embed_dim)
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [75]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    output, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    return output, final_state
   

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [76]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, rnn_size )
    output, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(output, vocab_size, activation_fn=None, 
                                               weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                              biases_initializer=tf.truncated_normal_initializer(stddev=0.1))
    return logits, final_state
    
  

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [77]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])
    ydata[-1] = xdata[0]

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

    #print(np.array(list(zip(x_batches, y_batches))))
        
    return np.array(list(zip(x_batches, y_batches)))
    
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.



- Enough epochs to get near a minimum in the training loss, no real upper limit on this. Just need to make sure the training loss is low and not improving much with more training.
- Batch size is large enough to train efficiently, but small enough to fit the data in memory. No real “best” value here, depends on GPU memory usually.
- Size of the RNN cells (number of units in the hidden layers) is large enough to fit the data well. Again, no real “best” value.
- The sequence length (seq_length) here should be about the size of the length of sentences you want to generate. Should match the structure of the data.
The learning rate shouldn’t be too large because the training algorithm won’t converge. But needs to be large enough that training doesn’t take forever.
Set show_every_n_batches to the number of batches the neural network should print progress.

In [78]:

# Number of Epochs
num_epochs = 512   #500   #300  #128
# Batch Size
batch_size = 128    #100    #256
# RNN Size
rnn_size = 256      #512     #256
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 16        #50     #10       #50     #32
# Learning Rate
learning_rate = 0.001    #0.002
# Show stats for every n number of batches
show_every_n_batches = 8

'''
num_epochs = 128
batch_size = 256
rnn_size = 256
embed_dim = 300
seq_length = 32
learning_rate = 0.01
show_every_n_batches = 8
'''

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [79]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [80]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/33   train_loss = 8.823
Epoch   0 Batch    8/33   train_loss = 8.568
Epoch   0 Batch   16/33   train_loss = 7.251
Epoch   0 Batch   24/33   train_loss = 6.635
Epoch   0 Batch   32/33   train_loss = 6.561
Epoch   1 Batch    7/33   train_loss = 6.312
Epoch   1 Batch   15/33   train_loss = 6.111
Epoch   1 Batch   23/33   train_loss = 6.130
Epoch   1 Batch   31/33   train_loss = 5.978
Epoch   2 Batch    6/33   train_loss = 5.953
Epoch   2 Batch   14/33   train_loss = 5.953
Epoch   2 Batch   22/33   train_loss = 5.953
Epoch   2 Batch   30/33   train_loss = 5.794
Epoch   3 Batch    5/33   train_loss = 5.800
Epoch   3 Batch   13/33   train_loss = 5.685
Epoch   3 Batch   21/33   train_loss = 5.674
Epoch   3 Batch   29/33   train_loss = 5.548
Epoch   4 Batch    4/33   train_loss = 5.430
Epoch   4 Batch   12/33   train_loss = 5.481
Epoch   4 Batch   20/33   train_loss = 5.406
Epoch   4 Batch   28/33   train_loss = 5.397
Epoch   5 Batch    3/33   train_loss = 5.247
Epoch   5 

Epoch  44 Batch   12/33   train_loss = 3.192
Epoch  44 Batch   20/33   train_loss = 3.144
Epoch  44 Batch   28/33   train_loss = 3.146
Epoch  45 Batch    3/33   train_loss = 3.007
Epoch  45 Batch   11/33   train_loss = 3.004
Epoch  45 Batch   19/33   train_loss = 3.066
Epoch  45 Batch   27/33   train_loss = 3.192
Epoch  46 Batch    2/33   train_loss = 3.082
Epoch  46 Batch   10/33   train_loss = 3.019
Epoch  46 Batch   18/33   train_loss = 3.079
Epoch  46 Batch   26/33   train_loss = 3.040
Epoch  47 Batch    1/33   train_loss = 2.982
Epoch  47 Batch    9/33   train_loss = 3.002
Epoch  47 Batch   17/33   train_loss = 3.039
Epoch  47 Batch   25/33   train_loss = 3.051
Epoch  48 Batch    0/33   train_loss = 2.930
Epoch  48 Batch    8/33   train_loss = 2.988
Epoch  48 Batch   16/33   train_loss = 2.995
Epoch  48 Batch   24/33   train_loss = 3.009
Epoch  48 Batch   32/33   train_loss = 2.946
Epoch  49 Batch    7/33   train_loss = 2.942
Epoch  49 Batch   15/33   train_loss = 2.922
Epoch  49 

Epoch  88 Batch   24/33   train_loss = 1.967
Epoch  88 Batch   32/33   train_loss = 1.883
Epoch  89 Batch    7/33   train_loss = 1.920
Epoch  89 Batch   15/33   train_loss = 1.870
Epoch  89 Batch   23/33   train_loss = 1.903
Epoch  89 Batch   31/33   train_loss = 1.877
Epoch  90 Batch    6/33   train_loss = 1.895
Epoch  90 Batch   14/33   train_loss = 1.875
Epoch  90 Batch   22/33   train_loss = 1.874
Epoch  90 Batch   30/33   train_loss = 1.960
Epoch  91 Batch    5/33   train_loss = 1.857
Epoch  91 Batch   13/33   train_loss = 1.920
Epoch  91 Batch   21/33   train_loss = 1.843
Epoch  91 Batch   29/33   train_loss = 1.852
Epoch  92 Batch    4/33   train_loss = 1.872
Epoch  92 Batch   12/33   train_loss = 1.921
Epoch  92 Batch   20/33   train_loss = 1.921
Epoch  92 Batch   28/33   train_loss = 1.875
Epoch  93 Batch    3/33   train_loss = 1.841
Epoch  93 Batch   11/33   train_loss = 1.799
Epoch  93 Batch   19/33   train_loss = 1.821
Epoch  93 Batch   27/33   train_loss = 1.958
Epoch  94 

Epoch 133 Batch    3/33   train_loss = 1.232
Epoch 133 Batch   11/33   train_loss = 1.244
Epoch 133 Batch   19/33   train_loss = 1.253
Epoch 133 Batch   27/33   train_loss = 1.366
Epoch 134 Batch    2/33   train_loss = 1.303
Epoch 134 Batch   10/33   train_loss = 1.223
Epoch 134 Batch   18/33   train_loss = 1.250
Epoch 134 Batch   26/33   train_loss = 1.221
Epoch 135 Batch    1/33   train_loss = 1.267
Epoch 135 Batch    9/33   train_loss = 1.347
Epoch 135 Batch   17/33   train_loss = 1.258
Epoch 135 Batch   25/33   train_loss = 1.253
Epoch 136 Batch    0/33   train_loss = 1.247
Epoch 136 Batch    8/33   train_loss = 1.264
Epoch 136 Batch   16/33   train_loss = 1.256
Epoch 136 Batch   24/33   train_loss = 1.285
Epoch 136 Batch   32/33   train_loss = 1.212
Epoch 137 Batch    7/33   train_loss = 1.238
Epoch 137 Batch   15/33   train_loss = 1.194
Epoch 137 Batch   23/33   train_loss = 1.191
Epoch 137 Batch   31/33   train_loss = 1.217
Epoch 138 Batch    6/33   train_loss = 1.228
Epoch 138 

Epoch 177 Batch   15/33   train_loss = 0.893
Epoch 177 Batch   23/33   train_loss = 0.846
Epoch 177 Batch   31/33   train_loss = 0.906
Epoch 178 Batch    6/33   train_loss = 0.908
Epoch 178 Batch   14/33   train_loss = 0.925
Epoch 178 Batch   22/33   train_loss = 0.859
Epoch 178 Batch   30/33   train_loss = 0.966
Epoch 179 Batch    5/33   train_loss = 0.870
Epoch 179 Batch   13/33   train_loss = 0.946
Epoch 179 Batch   21/33   train_loss = 0.816
Epoch 179 Batch   29/33   train_loss = 0.934
Epoch 180 Batch    4/33   train_loss = 0.908
Epoch 180 Batch   12/33   train_loss = 0.945
Epoch 180 Batch   20/33   train_loss = 0.933
Epoch 180 Batch   28/33   train_loss = 0.900
Epoch 181 Batch    3/33   train_loss = 0.844
Epoch 181 Batch   11/33   train_loss = 0.889
Epoch 181 Batch   19/33   train_loss = 0.895
Epoch 181 Batch   27/33   train_loss = 0.979
Epoch 182 Batch    2/33   train_loss = 0.926
Epoch 182 Batch   10/33   train_loss = 0.868
Epoch 182 Batch   18/33   train_loss = 0.877
Epoch 182 

...

Epoch 310 Batch   18/33   train_loss = 0.409
Epoch 310 Batch   26/33   train_loss = 0.400
Epoch 311 Batch    1/33   train_loss = 0.451
Epoch 311 Batch    9/33   train_loss = 0.456
Epoch 311 Batch   17/33   train_loss = 0.406
Epoch 311 Batch   25/33   train_loss = 0.397
Epoch 312 Batch    0/33   train_loss = 0.435
Epoch 312 Batch    8/33   train_loss = 0.407
Epoch 312 Batch   16/33   train_loss = 0.431
Epoch 312 Batch   24/33   train_loss = 0.408
Epoch 312 Batch   32/33   train_loss = 0.427
Epoch 313 Batch    7/33   train_loss = 0.447
Epoch 313 Batch   15/33   train_loss = 0.385
Epoch 313 Batch   23/33   train_loss = 0.387
Epoch 313 Batch   31/33   train_loss = 0.398
Epoch 314 Batch    6/33   train_loss = 0.417
Epoch 314 Batch   14/33   train_loss = 0.437
Epoch 314 Batch   22/33   train_loss = 0.404
Epoch 314 Batch   30/33   train_loss = 0.420
Epoch 315 Batch    5/33   train_loss = 0.404
Epoch 315 Batch   13/33   train_loss = 0.436
Epoch 315 Batch   21/33   train_loss = 0.375
Epoch 315 

Epoch 354 Batch   30/33   train_loss = 0.346
Epoch 355 Batch    5/33   train_loss = 0.350
Epoch 355 Batch   13/33   train_loss = 0.372
Epoch 355 Batch   21/33   train_loss = 0.338
Epoch 355 Batch   29/33   train_loss = 0.377
Epoch 356 Batch    4/33   train_loss = 0.338
Epoch 356 Batch   12/33   train_loss = 0.373
Epoch 356 Batch   20/33   train_loss = 0.381
Epoch 356 Batch   28/33   train_loss = 0.359
Epoch 357 Batch    3/33   train_loss = 0.337
Epoch 357 Batch   11/33   train_loss = 0.379
Epoch 357 Batch   19/33   train_loss = 0.383
Epoch 357 Batch   27/33   train_loss = 0.392
Epoch 358 Batch    2/33   train_loss = 0.372
Epoch 358 Batch   10/33   train_loss = 0.364
Epoch 358 Batch   18/33   train_loss = 0.340
Epoch 358 Batch   26/33   train_loss = 0.334
Epoch 359 Batch    1/33   train_loss = 0.381
Epoch 359 Batch    9/33   train_loss = 0.393
Epoch 359 Batch   17/33   train_loss = 0.350
Epoch 359 Batch   25/33   train_loss = 0.327
Epoch 360 Batch    0/33   train_loss = 0.355
Epoch 360 

Epoch 399 Batch    9/33   train_loss = 0.342
Epoch 399 Batch   17/33   train_loss = 0.318
Epoch 399 Batch   25/33   train_loss = 0.315
Epoch 400 Batch    0/33   train_loss = 0.331
Epoch 400 Batch    8/33   train_loss = 0.327
Epoch 400 Batch   16/33   train_loss = 0.324
Epoch 400 Batch   24/33   train_loss = 0.312
Epoch 400 Batch   32/33   train_loss = 0.335
Epoch 401 Batch    7/33   train_loss = 0.344
Epoch 401 Batch   15/33   train_loss = 0.325
Epoch 401 Batch   23/33   train_loss = 0.312
Epoch 401 Batch   31/33   train_loss = 0.315
Epoch 402 Batch    6/33   train_loss = 0.330
Epoch 402 Batch   14/33   train_loss = 0.339
Epoch 402 Batch   22/33   train_loss = 0.319
Epoch 402 Batch   30/33   train_loss = 0.307
Epoch 403 Batch    5/33   train_loss = 0.316
Epoch 403 Batch   13/33   train_loss = 0.350
Epoch 403 Batch   21/33   train_loss = 0.298
Epoch 403 Batch   29/33   train_loss = 0.327
Epoch 404 Batch    4/33   train_loss = 0.317
Epoch 404 Batch   12/33   train_loss = 0.330
Epoch 404 

Epoch 443 Batch   21/33   train_loss = 0.302
Epoch 443 Batch   29/33   train_loss = 0.325
Epoch 444 Batch    4/33   train_loss = 0.296
Epoch 444 Batch   12/33   train_loss = 0.302
Epoch 444 Batch   20/33   train_loss = 0.333
Epoch 444 Batch   28/33   train_loss = 0.299
Epoch 445 Batch    3/33   train_loss = 0.286
Epoch 445 Batch   11/33   train_loss = 0.314
Epoch 445 Batch   19/33   train_loss = 0.313
Epoch 445 Batch   27/33   train_loss = 0.337
Epoch 446 Batch    2/33   train_loss = 0.298
Epoch 446 Batch   10/33   train_loss = 0.292
Epoch 446 Batch   18/33   train_loss = 0.283
Epoch 446 Batch   26/33   train_loss = 0.277
Epoch 447 Batch    1/33   train_loss = 0.344
Epoch 447 Batch    9/33   train_loss = 0.311
Epoch 447 Batch   17/33   train_loss = 0.292
Epoch 447 Batch   25/33   train_loss = 0.287
Epoch 448 Batch    0/33   train_loss = 0.298
Epoch 448 Batch    8/33   train_loss = 0.295
Epoch 448 Batch   16/33   train_loss = 0.318
Epoch 448 Batch   24/33   train_loss = 0.277
Epoch 448 

Epoch 488 Batch    0/33   train_loss = 0.275
Epoch 488 Batch    8/33   train_loss = 0.301
Epoch 488 Batch   16/33   train_loss = 0.295
Epoch 488 Batch   24/33   train_loss = 0.280
Epoch 488 Batch   32/33   train_loss = 0.299
Epoch 489 Batch    7/33   train_loss = 0.309
Epoch 489 Batch   15/33   train_loss = 0.273
Epoch 489 Batch   23/33   train_loss = 0.275
Epoch 489 Batch   31/33   train_loss = 0.286
Epoch 490 Batch    6/33   train_loss = 0.292
Epoch 490 Batch   14/33   train_loss = 0.325
Epoch 490 Batch   22/33   train_loss = 0.287
Epoch 490 Batch   30/33   train_loss = 0.275
Epoch 491 Batch    5/33   train_loss = 0.293
Epoch 491 Batch   13/33   train_loss = 0.297
Epoch 491 Batch   21/33   train_loss = 0.274
Epoch 491 Batch   29/33   train_loss = 0.306
Epoch 492 Batch    4/33   train_loss = 0.276
Epoch 492 Batch   12/33   train_loss = 0.286
Epoch 492 Batch   20/33   train_loss = 0.309
Epoch 492 Batch   28/33   train_loss = 0.286
Epoch 493 Batch    3/33   train_loss = 0.272
Epoch 493 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [81]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [82]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [83]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    probs_tensor = loaded_graph.get_tensor_by_name("probs:0")
    return input_tensor, initial_state_tensor, final_state_tensor, probs_tensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [84]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    return np.random.choice(list(int_to_vocab.values()),1, list(probabilities))[0]
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [85]:
gen_length = 256
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'homer_simpson'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
homer_simpson: material reserved online civic yap aah upset well penny butterball friend: occupation pronounce swine united glum admitting guys faceful saving espousing sister fans'll sideshow mocking answer tie duff_announcer: frink ten babar safely space-time each remorseful tinklin' briefly happy court statue eventually yourselves charm conversation money's falling neighbor naively phone's waylon_smithers: satisfaction exhale hillary anything england's queen eyesore mckinley cannoli rueful are appointment yelling dregs leave twice mural twins people alright muttering detail sobriety pridesters: toss all bunion conclude phase amanda washouts wiping ehhhhhhhhh belt solved crow sistine puttin' annual drains enforced leprechaun bronco buried smokes sly lachrymose crinkly terrible formico charming taxi plenty generous supervising monkeyshines background african obvious treats judge polygon thing goes time's bowie pretzels box cure honored 

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.